# Vamos aprender a trabalhar com PDF usando o Python

- Regra geral: PDF foi feito justamente para bloquear muita coisa, então não é fácil "brincar" com um pdf
- Mesmo assim, Python tem várias bibliotecas que vão nos ajudar, vamos focar em 2:
    - PyPDF2
    - Tabula
- Ler e extrair informações de um PDF a gente consegue fazer.
- Escrever e Editar, aí já é outra história

### Para os nossos exemplos, vamos avaliar o Release de Resultados do 3º e 4º Trimestre de 2020 da Magazine Luiza

#### 1º Objetivo: Queremos conseguir separar apenas o DRE do Release de Resultados (Página 14) para enviar para a Diretoria, como fazemos?
    - Separar as páginas de um pdf

In [9]:
from pathlib import Path
import PyPDF2 as pyf

CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados')
for arquivo in CAMINHO_REFERENCIA.iterdir():
    if arquivo.is_file():
        arquivo_PDF = pyf.PdfReader(arquivo)
        for pos, page in enumerate(arquivo_PDF.pages, 1):
            novo_PDF = pyf.PdfWriter()
            novo_PDF.add_page(page)
            with open(CAMINHO_REFERENCIA.joinpath(f"paginas/{arquivo.name.removesuffix(arquivo.suffix)} - {pos}{arquivo.suffix}"), 'wb') as novo_arquivo:
                novo_PDF.write(novo_arquivo)


#### 2º Objetivo: Com o Release de Resultados já separado página por página, queremos incluir apenas as Páginas de Destaque (Página 1), DRE (Página 14) e Balanço (Página 16).
    - Juntar vários pdfs em 1

In [10]:
from pathlib import Path
import PyPDF2 as pyf

CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados/paginas')
num_paginas = [1, 14, 16]
novo_PDF = pyf.PdfWriter()
nome_referencia = 'MGLU_ER_3T20_POR'
paginas = ''
for numero in num_paginas:
    arquivo_PDF = pyf.PdfReader(CAMINHO_REFERENCIA.joinpath(f'{nome_referencia} - {numero}.pdf'))
    novo_PDF.add_page(arquivo_PDF.pages[0])
    paginas += f'{numero}_'
if len(paginas) > 0:
    paginas = paginas[:-1]
with open(CAMINHO_REFERENCIA.joinpath(f"{nome_referencia} - {paginas}.pdf"), 'wb') as novo_arquivo:
                novo_PDF.write(novo_arquivo)

### Extra: Para adicionar todas as páginas de 2 pdfs

In [12]:
from pathlib import Path
import PyPDF2 as pyf

pdf_mesclado = pyf.PdfMerger()
CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados')
for arquivo in CAMINHO_REFERENCIA.iterdir():
    if arquivo.is_file():
        pdf_mesclado.append(arquivo)
with open(CAMINHO_REFERENCIA.joinpath(f"Arquivo Mesclado.pdf"), 'wb') as novo_arquivo:
    pdf_mesclado.write(novo_arquivo)


# Funcionalidades que podem ser úteis:

- Inserir arquivo no meio do outro
- Quero colocar dentro do Resultado do 4T20 os destaques do 3T20 para poder comparar os 2 dentro do mesmo relatório

In [2]:
from pathlib import Path
import PyPDF2 as pyf

pdf_mesclado = pyf.PdfMerger()
CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados')
pdf_mesclado.append(CAMINHO_REFERENCIA.joinpath('MGLU_ER_4T20_POR.pdf'))
pdf_mesclado.merge(1, CAMINHO_REFERENCIA.joinpath('paginas/MGLU_ER_3T20_POR - 1.pdf'))

with open(CAMINHO_REFERENCIA.joinpath(f"Relatorio 2 Trimestres.pdf"), 'wb') as novo_arquivo:
    pdf_mesclado.write(novo_arquivo)


- Rodar Página

In [2]:
from pathlib import Path
import PyPDF2 as pyf

CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados')
arquivo_PDF = pyf.PdfReader(CAMINHO_REFERENCIA.joinpath(f"MGLU_ER_3T20_POR.pdf"))
novo_PDF = pyf.PdfWriter()
for page in arquivo_PDF.pages:
    page.rotate(90)
    novo_PDF.add_page(page)
with open(CAMINHO_REFERENCIA.joinpath(f"Paginas Rotacionadas.pdf"), 'wb') as novo_arquivo:
    novo_PDF.write(novo_arquivo)


# Trabalhando com Textos e Informações Dentro do PDF

#### 1º Objetivo: Quero identificar como foram as Despesas com Vendas da MGLU
    - Pegar texto da página e identificar onde está essa informação

In [20]:
from pathlib import Path
import PyPDF2 as pyf

texto_referencia = "| Despesas com Vendas"

CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados')
arquivo_PDF = pyf.PdfReader(CAMINHO_REFERENCIA.joinpath(f"MGLU_ER_3T20_POR.pdf"))

for pos, pagina in enumerate(arquivo_PDF.pages, 1):
    texto_pagina = pagina.extract_text()
    if texto_referencia in texto_pagina:
        texto_analisar = texto_pagina

inicio_texto = texto_analisar.find(texto_referencia)
final_texto = texto_analisar.find("|", )
print(texto_analisar[inicio_texto:final_texto])




| Despesas com Vendas  
 
No 3T20, as despesas com vendas totalizaram R$1.432,6 milhões, equivalentes a 17,2% da receita líquida, 1,1 p.p. menor que no 
3T19 , principalmente devido ao forte crescimento das vendas . Vale ressaltar que a Companhia conseguiu diluir as despesas com 
vendas m esmo investi ndo em maior nível de serviço,  especialmente em  atendimento e logística.  
 
Nos 9M20, as despesas com vendas totalizaram R$3.487,2 milhões, equivalentes a 18,2% da receita líquida (+1,1 p.p. versus  os 
9M19).  
 
| Despesas Gerais e Administrativas   
 
No 3T20, as despesas gerais e administrativas totalizaram R$240,7 milhões, equivalentes a 2,9% da receita líquida, uma reduçã o 
de 1,4 p.p. em relação ao 3T19 devido, principalm ente, ao forte crescimento das vendas.   
 
Nos 9M20, as despesas gerais e admini strativas totalizaram R$617,3 milhões, equivalentes a 3,2% da receita líquida.  
 
| Perdas em Créditos de Liquidação Duvidosa  
 
As provisões para perdas em crédito de liquidaç

#### 2º Objetivo: Quero analisar o DRE (sem ajuste - Página 5)
    - Para ler tabelas em pdf, use o tabula (é ninja)
    
    - Cuidado 1: Instale o tabula-py (não instale o tabula). Se instalar o tabula errado, desinstale ele, instale o tabula-py, desinstale o tabula-py e instale novamente o tabula-py. Reinicie o kernel do Jupyter após isso
    
    - Cuidado 2: Tem que ter o java instalado no seu computador (depois de instalar, reinicie o computador)

#### 3º Objetivo: Quero analisar o Capital de Giro e os Investimentos (ambas as tabelas na página 12)
    - Páginas com mais de 1 tabela

#### O que fazer quando o tabula não consegue ler alguma linha da tabela? Como o cabeçalho, no nosso caso?

# Outro método que pode ser útil algum dia: Captar Imagem em um pdf
    - biblioteca pikepdf

# Substituir texto no pdf tipo contrato

- Não recomendo fazer diretamente pelo Python. Realmente do que vi a melhor opção me parece o Word fazer isso
- Caso precise automatizar, automatize o processo fazendo ele pelo Word
- Quem quiser MUITO fazer isso pelo Python, tem um link aqui que vai te ajudar de uma solução que achei que funciona. Tem seus bugs/cuidados especiais, mas funciona: https://pdf.co/samples/pdf-co-web-api-replace-text-from-pdf-python-replace-text-from-url